In [ ]:
# load libraries
import pandas as pd
import janitor
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from importlib.metadata import version

print("Pandas version: " + str(pd.__version__))
print("Janitor version: " + str(janitor.__version__))
print("Numpy version: " + str(np.__version__))
print("Matplotlib version: " + str(version("matplotlib")))
print("Seborn version: " + str(sns.__version__))
print("Fastparquet version: " + str(version("fastparquet")))

### About the data

These data come from the **New York State Department of Health**’s [Hospital Inpatient Discharges (SPARCS De-Identified)](https://health.data.ny.gov/Health/Hospital-Inpatient-Discharges-SPARCS-De-Identified/sf4k-39ay/about_data) dataset.
They represent hospital discharge records and include information on patient characteristics, diagnoses, treatments, services, and charges.
- **Scope:** 2,196,737 total discharges across New York State
- **Variables:** 33 columns
- **Time Period:** Discharges from calendar year 2024
- **Last Updated:** October 10, 2025

For this project, the dataset is restricted to **influenza diagnoses only**, resulting in **7,213 records**.

### Data de-identification and redactions
All identifiable elements have been removed. Specific redactions include:
- **Abortion discharges:** Excluded from facility-level identifies (no Permanent Facility Id, Facility Name, Health Service Area, Hospital County, or Operating Certificate information).
- **ZIP code:** Limited to the first three digits.
  - ZIP codes are blank if:
    - The ZIP code area has fewer than 20,000 residents,
    - The cell size for patient classification strata is less than 10, or
    - The record represents an abortion discharge.
  - “OOS” ZIP code indicates *Out-of-State* ZIP codes.
- **Age:** Reported in broad categories: *0 to 17, 18 to 29, 30 to 49, 50 to 69, and 70+ years*

In [ ]:
# define column data types
dtype_map = {
    "Health Service Area": "string",
    "Hospital County": "string",
    "Operating Certificate Number": "string",
    "Permanent Facility Id": "string",
    "Facility Name": "string",
    "Age Group": "string",
    "Zip Code": "string",
    "Gender": "string",
    "Race": "string",
    "Ethnicity": "string",
    "Length of Stay": "string",
    "Type of Admission": "string",
    "Patient Disposition": "string",
    "Discharge Year": "string",
    "CCSR Diagnosis Code": "string",
    "CCSR Diagnosis Description": "string",
    "CCSR Procedure Code": "string",
    "CCSR Procedure Description": "string",
    "APR DRG Code": "string",
    "APR DRG Description": "string",
    "APR MDC Code": "string",
    "APR MDC Description": "string",
    "APR Severity of Illness Code": "string",
    "APR Severity of Illness Description": "string",
    "APR Risk of Mortality": "string",
    "APR Medical Surgical Description": "string",
    "Payment Typology 1": "string",
    "Payment Typology 2": "string",
    "Payment Typology 3": "string",
    "Birth Weight": "string",
    "Emergency Department Indicator": "string",
    "Total Charges": "float64",
    "Total Costs": "float64"
}

In [ ]:
# read in data, clean names, subset, and print first 5 rows
df = pd.read_csv('../data/untouched/Hospital_Inpatient_Discharges_(SPARCS_De-Identified)__2024_20251106.csv', dtype = dtype_map)

df = df.clean_names()

df = df[df['ccsr_diagnosis_description'] == 'INFLUENZA'].reset_index(drop = True)

df.head()

In [ ]:
# data set information
df.info()

In [ ]:
# summary of all columns
df.describe(include = 'all').T

In [ ]:
# missing values
df.isna().sum()

In [ ]:
categorical_cols = ['health_service_area', 'hospital_county', 'operating_certificate_number', 
                    'permanent_facility_id', 'facility_name', 'age_group', 
                    'zip_code', 'gender', 'race', 
                    'ethnicity', 'type_of_admission', 'patient_disposition', 
                    'discharge_year', 'ccsr_diagnosis_code', 'ccsr_diagnosis_description', 
                    'ccsr_procedure_code', 'ccsr_procedure_description', 'apr_drg_code', 
                    'apr_drg_description', 'apr_mdc_code', 'apr_mdc_description', 
                    'apr_severity_of_illness_code', 'apr_severity_of_illness_description', 'apr_risk_of_mortality', 
                    'apr_medical_surgical_description', 'payment_typology_1', 'payment_typology_2', 
                    'payment_typology_3', 'emergency_department_indicator']

numeric_cols = ['length_of_stay', 'birth_weight', 'total_charges', 
                'total_costs']

In [ ]:
for col in categorical_cols:
    print(f"--- {col} ---")
    freq = df[col].value_counts(dropna = False)
    percent = df[col].value_counts(normalize = True, dropna = False) * 100
    summary = pd.DataFrame({'Count': freq, 'Percent': percent.round(2)})
    print(summary, "\n")

In [ ]:
# cross-tab of zip_code and health_service_area
pd.crosstab(df['zip_code'], df['health_service_area'])

In [ ]:
# check if facility_id is nested in operating_certificate_number
facility_to_cert = df.groupby('permanent_facility_id')['operating_certificate_number'].nunique()
print(f"Facility IDs with multiple certificate numbers: {(facility_to_cert > 1).sum()}")
print(f"Facility IDs with one certificate number: {(facility_to_cert == 1).sum()}")

# check the reverse - do certificate numbers span multiple facilities?
cert_to_facility = df.groupby('operating_certificate_number')['permanent_facility_id'].nunique()
print(f"\nCertificate numbers with multiple facility IDs: {(cert_to_facility > 1).sum()}")
print(f"Certificate numbers with one facility ID: {(cert_to_facility == 1).sum()}")

# check facility_id and facility_name relationship
facility_to_name = df.groupby('permanent_facility_id')['facility_name'].nunique()
print(f"\nFacility IDs with multiple names: {(facility_to_name > 1).sum()}")
print(f"Facility IDs with one name: {(facility_to_name == 1).sum()}")

name_to_facility = df.groupby('facility_name')['permanent_facility_id'].nunique()
print(f"\nFacility names with multiple IDs: {(name_to_facility > 1).sum()}")
print(f"Facility names with one ID: {(name_to_facility == 1).sum()}")

In [ ]:
# facility names with more than 1 facility id
df.groupby('facility_name')['permanent_facility_id'].nunique().loc[lambda x: x > 1]

In [ ]:
# check for pairs of columns that are redundent
columns = [col for col in df.columns if col not in ['discharge_year', 'ccsr_diagnosis_code', 'ccsr_diagnosis_description']]
redundant_pairs = []

for i, col1 in enumerate(columns):
    for col2 in columns[i+1:]:
        mapping = df[[col1, col2]].dropna().drop_duplicates()
        if mapping[col1].is_unique and mapping[col2].is_unique:
            redundant_pairs.append((col1, col2))

print("Columns with 1-to-1 mapping:", redundant_pairs)

In [ ]:
df_clean = df.copy()

# replace missing facility characteristics with "NAA" (i.e., N/A abortion-related)
missing_cols_abortion = ['health_service_area', 'hospital_county', 'operating_certificate_number', 
                         'permanent_facility_id', 'zip_code']

df_clean.loc[df_clean['facility_name'] == 'Redacted for Confidentiality', missing_cols_abortion] = (
    df_clean.loc[df_clean['facility_name'] == 'Redacted for Confidentiality', missing_cols_abortion].fillna('NAA')
)

# replace other missing zip_code values with "NAS" (i.e., N/A small sample)
df_clean.loc[df_clean['facility_name'] != 'Redacted for Confidentiality', 'zip_code'] = \
    df_clean.loc[df_clean['facility_name'] != 'Redacted for Confidentiality', 'zip_code'].fillna('NAS')

# convert 120+ length_of_stay values to 120
df_clean['length_of_stay'] = df_clean['length_of_stay'].replace('120+', 120).astype('int64')

# payment_typology_2, payment_typology_3, ccsr_procedure_description
df_clean['payment_typology_2'] = df_clean['payment_typology_2'].fillna("None")
df_clean['payment_typology_3'] = df_clean['payment_typology_3'].fillna("None")
df_clean['ccsr_procedure_description'] =df_clean['ccsr_procedure_description'].fillna("None")

In [ ]:
"""drop redundent and unusable columns
   - operating_certificate_number and facility_name are redundant with permanent_facility_id and not as granular, keep permanent_facility_id only
   - discharge_year, ccsr_diagnosis_code, and ccsr_diagnosis_description have no variation (only 1 value)
   - ccsr_procedure_code, apr_drg_code, apr_mdc_code, and apr_severity_of_illness_code are redundent
   - birth_weight is 99.5% missing
   - total_charges and total_costs will be dropped because they would not be known during the visit and are partially derived from length of stay
"""
df_clean.drop(['operating_certificate_number', 'facility_name', 'discharge_year', 
               'ccsr_procedure_code', 'apr_drg_code', 'apr_mdc_code', 'apr_severity_of_illness_code', 
               'ccsr_diagnosis_code', 'ccsr_diagnosis_description', 'birth_weight', 
               'total_charges', 'total_costs'], axis = 1, inplace = True)

In [ ]:
# check missing values again
df_clean.isna().sum()

In [ ]:
# new columns
categorical_cols = ['health_service_area', 'hospital_county', 'permanent_facility_id', 
                    'age_group', 'zip_code', 'gender', 
                    'race', 'ethnicity', 'type_of_admission', 
                    'patient_disposition', 'ccsr_procedure_description', 'apr_drg_description',
                    'apr_mdc_description', 'apr_severity_of_illness_description', 'apr_risk_of_mortality',
                    'apr_medical_surgical_description', 'payment_typology_1', 'payment_typology_2', 
                    'payment_typology_3', 'emergency_department_indicator']

numeric_cols = ['length_of_stay']

In [ ]:
# run frequencies again
for col in categorical_cols:
    print(f"--- {col} ---")
    freq = df_clean[col].value_counts(dropna = False)
    percent = df_clean[col].value_counts(normalize = True, dropna = False) * 100
    summary = pd.DataFrame({'Count': freq, 'Percent': percent.round(2)})
    print(summary, '\n')

In [ ]:
df_clean.info()

In [ ]:
# histograms for continuous variables
for col in numeric_cols:
    plt.figure(figsize = (8, 4))
    sns.histplot(df_clean[col], kde = True, bins = 50)
    plt.title(f'{col} Distribution')
    plt.xlabel(col)
    plt.ylabel('Frequency')
    plt.show()

In [ ]:
# boxplots for continuous variables
for col in numeric_cols:
    plt.figure(figsize = (6,4))
    sns.boxplot(x = df_clean[col])
    plt.title(f'{col} - Boxplot')
    plt.show()

In [ ]:
# output for further analysis
df_clean.to_parquet('../data/temp/nystate_flu_discharges_2024.parquet', engine = "fastparquet", index = False)